# Phase 1: Data Cleaning and Preprocessing

In [59]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler

#Loading the dataset
dataset_file_path = "C:\\Users\\TazeenQ\\team27_project\\data\\raw\\retail_sales_dataset.csv"
retail_data = pd.read_csv(dataset_file_path)

#Displaying the first couple of rows to check the data
retail_data.head(), retail_data.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 9 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   Transaction ID    1000 non-null   int64 
 1   Date              1000 non-null   object
 2   Customer ID       1000 non-null   object
 3   Gender            1000 non-null   object
 4   Age               1000 non-null   int64 
 5   Product Category  1000 non-null   object
 6   Quantity          1000 non-null   int64 
 7   Price per Unit    1000 non-null   int64 
 8   Total Amount      1000 non-null   int64 
dtypes: int64(5), object(4)
memory usage: 70.4+ KB


(   Transaction ID        Date Customer ID  Gender  Age Product Category  \
 0               1  2023-11-24     CUST001    Male   34           Beauty   
 1               2  2023-02-27     CUST002  Female   26         Clothing   
 2               3  2023-01-13     CUST003    Male   50      Electronics   
 3               4  2023-05-21     CUST004    Male   37         Clothing   
 4               5  2023-05-06     CUST005    Male   30           Beauty   
 
    Quantity  Price per Unit  Total Amount  
 0         3              50           150  
 1         2             500          1000  
 2         1              30            30  
 3         1             500           500  
 4         2              50           100  ,
 None)

In [ ]:
#Renaming column names
retail_data.rename(columns = lambda x: x.strip().replace(" ", "_").lower(), inplace = True)

#Standardizing data types
#Converting date to datetime
retail_data['date'] = pd.to_datetime(retail_data['date'], errors = 'coerce')

#Handling missing values by replacing blanks with NaN and checking for missing values
retail_data.replace("", np.nan, inplace = True)
missing_summary = retail_data.isnull().sum()

#Dropping rows with date missing
retail_data.dropna(subset = ['date'], inplace = True)

#Filtering data, the dataset has values from 2023 and only 2 rows with data from 2024. I am removing those 2 rows.
retail_data = retail_data[retail_data['date'].dt.year != 2024].reset_index(drop = True)

#Summarizing dataset features
summary_stats = retail_data.describe(include = 'all')

#Choosing numerical columns to normalize
numerical_cols = ['age', 'quantity', 'price_per_unit', 'total_amount']

#Normalizng them
scaler = MinMaxScaler()
normalized_data = scaler.fit_transform(retail_data[numerical_cols])

#Creating a new DataFrame for the normalized data
normalized_dataset = retail_data.copy()
normalized_dataset[numerical_cols] = pd.DataFrame(normalized_data, columns = numerical_cols)

#Hot encoding my categorical variables
categorical_columns = ['gender', 'product_category']
encoded_date = pd.get_dummies(normalized_dataset, columns = categorical_columns, drop_first = False)

#Saving the normalized dataset to a new file
cleaned_dataset_file_path = "C:\\Users\\TazeenQ\\team27_project\\data\\processed\\processed_retail_sales_dataset.csv"
normalized_dataset.to_csv(cleaned_dataset_file_path, index = True) #true for logistic regression (drop beauty category), false for KNN and random forests - multicollinearity

normalized_dataset.info(), normalized_dataset.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 998 entries, 0 to 997
Data columns (total 9 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   transaction_id    998 non-null    int64         
 1   date              998 non-null    datetime64[ns]
 2   customer_id       998 non-null    object        
 3   gender            998 non-null    object        
 4   age               998 non-null    float64       
 5   product_category  998 non-null    object        
 6   quantity          998 non-null    float64       
 7   price_per_unit    998 non-null    float64       
 8   total_amount      998 non-null    float64       
dtypes: datetime64[ns](1), float64(4), int64(1), object(3)
memory usage: 70.3+ KB


(None,
    transaction_id       date customer_id  gender       age product_category  \
 0               1 2023-11-24     CUST001    Male  0.347826           Beauty   
 1               2 2023-02-27     CUST002  Female  0.173913         Clothing   
 2               3 2023-01-13     CUST003    Male  0.695652      Electronics   
 3               4 2023-05-21     CUST004    Male  0.413043         Clothing   
 4               5 2023-05-06     CUST005    Male  0.260870           Beauty   
 
    quantity  price_per_unit  total_amount  
 0  0.666667        0.052632      0.063291  
 1  0.333333        1.000000      0.493671  
 2  0.000000        0.010526      0.002532  
 3  0.000000        1.000000      0.240506  
 4  0.333333        0.052632      0.037975  )

# Phase 2: Applying Statistical Methods